In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
sst_col_list = [c for c in add_train.columns if 'sst' in c]
sst_col_list

['sst-2010-1',
 'sst-2010-2',
 'sst-2010-3',
 'sst-2010-4',
 'sst-2010-5',
 'sst-2010-6',
 'sst-2010-7',
 'sst-2010-8',
 'sst-2010-9',
 'sst-2010-10']

In [4]:
train_df = pd.read_csv('./fe_v2_train_076.csv')
test_df = pd.read_csv('./fe_v2_test_076.csv')

In [5]:
exclude_cols = ['index', 'startdate']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 129


In [7]:
group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']

In [8]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5']

In [9]:
print("Len:", len(best_cols))

Len: 116


In [10]:
def assign_train_test_df(df, my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()
    return df

def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

def create_feature(df, alias_name, col_name, suffix_num, season_group_cols):
    first_df = create_df_by_group_and_agg(df, season_group_cols, "first", col_name, f"first_{alias_name}_{suffix_num}")
    min_df = create_df_by_group_and_agg(df, season_group_cols, "min", col_name, f"min_{alias_name}_{suffix_num}")
    max_df = create_df_by_group_and_agg(df, season_group_cols, "max", col_name, f"max_{alias_name}_{suffix_num}")
    
    df = df.merge(first_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_first'] = df[col_name] - df[f"first_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_first')
    
    df = df.merge(min_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_min'] = df[col_name] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_min')
    
    df = df.merge(max_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_max'] = df[col_name] - df[f"max_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_max')
    
    df[f'range_{alias_name}_{suffix_num}'] = df[f"max_{alias_name}_{suffix_num}"] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'range_{alias_name}_{suffix_num}')
    
    df[f'scale_{alias_name}_{suffix_num}'] = (df[col_name] - df[f"min_{alias_name}_{suffix_num}"]) / df[f'range_{alias_name}_{suffix_num}']
    df = assign_train_test_df(df, f'scale_{alias_name}_{suffix_num}')
    
    return [df, f'diff_{alias_name}_{suffix_num}_first',
   f'diff_{alias_name}_{suffix_num}_min',
    f'diff_{alias_name}_{suffix_num}_max',
    f'range_{alias_name}_{suffix_num}', f'scale_{alias_name}_{suffix_num}']

In [11]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_10 = 'contest-wind-h10-14d__wind-hgt-10'
sst_1_col = 'sst-2010-1'
sst_5_col = 'sst-2010-5'
sst_3_col = 'sst-2010-3'
sst_4_col = 'sst-2010-4'
sst_6_col = 'sst-2010-6'
sst_7_col = 'sst-2010-7'
sst_8_col = 'sst-2010-8'
sst_9_col = 'sst-2010-9'
sst_10_col = 'sst-2010-10'

In [12]:
train_df[sst_col_list] = add_train[sst_col_list]
test_df[sst_col_list] = add_test[sst_col_list]

In [13]:
df = pd.concat([train_df[main_attrs], test_df[main_attrs]], axis=0)
df.shape

(407088, 129)

In [14]:
pres_res = create_feature(df, 'pres', pres_col, 1, season_group_cols)
df = pres_res[0]
pres_feature = pres_res[1:]

Nan
Nan
Nan


In [15]:
slp_res = create_feature(df, 'slp', slp_col, 1, season_group_cols)
df = slp_res[0]
slp_feature = slp_res[1:]

Nan
Nan
Nan


In [16]:
precip_res = create_feature(df, 'precip', precip_col, 1, season_group_cols)
df = precip_res[0]
precip_feature = precip_res[1:]

Nan
Nan
Nan


In [17]:
sst_1_month_res = create_feature(df, 'sst_1_month', sst_1_col, 1, group_cols)
df = sst_1_month_res[0]
sst_1_month_feature = sst_1_month_res[1:]

Nan
Nan
Nan


In [18]:
sst_5_month_res = create_feature(df, 'sst_5_month', sst_5_col, 1, group_cols)
df = sst_5_month_res[0]
sst_5_month_feature = sst_5_month_res[1:]

Nan
Nan
Nan


In [19]:
sst_3_month_res = create_feature(df, 'sst_3_month', sst_3_col, 1, group_cols)
df = sst_3_month_res[0]
sst_3_month_feature = sst_3_month_res[1:]

Nan
Nan
Nan


In [20]:
sst_4_month_res = create_feature(df, 'sst_4_month', sst_4_col, 1, group_cols)
df = sst_4_month_res[0]
sst_4_month_feature = sst_4_month_res[1:]

Nan
Nan
Nan


In [21]:
sst_6_month_res = create_feature(df, 'sst_6_month', sst_6_col, 1, group_cols)
df = sst_6_month_res[0]
sst_6_month_feature = sst_6_month_res[1:]

Nan
Nan
Nan


In [22]:
wind_10_res = create_feature(df, 'wind_10', wind_col_10, 1, season_group_cols)
df = wind_10_res[0]
wind_10_feature = wind_10_res[1:]

Nan
Nan
Nan


In [23]:
wind_10_res_month = create_feature(df, 'wind_10_month', wind_col_10, 1, group_cols)
df = wind_10_res_month[0]
wind_10_month_feature = wind_10_res_month[1:]

Nan
Nan
Nan


In [24]:
modified_best_cols = [c for c in best_cols]

In [42]:
# df['sst_1_lag_1'] = df.groupby(group_cols)[sst_1_col].shift(1)
# df = assign_train_test_df(df, 'sst_1_lag_1')

# df['sst_1_lag_2'] = df.groupby(group_cols)[sst_1_col].shift(2)
# df = assign_train_test_df(df, 'sst_1_lag_2')

# df['sst_1_lag_3'] = df.groupby(group_cols)[sst_1_col].shift(3)
# df = assign_train_test_df(df, 'sst_1_lag_3')

df['sst_5_lag_1'] = df.groupby(group_cols)[sst_5_col].shift(1)
df = assign_train_test_df(df, 'sst_5_lag_1')

In [43]:
features = modified_best_cols + pres_feature + sst_1_month_feature + ['sst_1_lag_1', 'sst_5_lag_1']

In [45]:
# features

In [46]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v7.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:08,  1.74s/it]


(375734, 128) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 6.42ms	remaining: 32.1s
200:	learn: 1.1674178	test: 1.1730383	best: 1.1730383 (200)	total: 1.08s	remaining: 25.9s
400:	learn: 0.9139139	test: 0.9227852	best: 0.9227852 (400)	total: 2.13s	remaining: 24.4s
600:	learn: 0.7886693	test: 0.7993508	best: 0.7993508 (600)	total: 3.2s	remaining: 23.4s
800:	learn: 0.6999593	test: 0.7126921	best: 0.7126921 (800)	total: 4.21s	remaining: 22.1s
1000:	learn: 0.6447686	test: 0.6590992	best: 0.6590992 (1000)	total: 5.22s	remaining: 20.9s
1200:	learn: 0.5998430	test: 0.6154647	best: 0.6154647 (1200)	total: 6.33s	remaining: 20s
1400:	learn: 0.5629173	test: 0.5797005	best: 0.5797005 (1400)	total: 7.43s	remaining: 19.1s
1600:	learn: 0.5344115	test: 0.5519604	best: 0.5519604 (1600)	total: 8.43s	remaining: 17.9s
1800:	learn: 0.5096265	test: 0.5282609	best: 0.5282609 (1800)	total: 9.52s	remaining: 16.9s
2000:	learn: 0.4872345	test: 0.5065798	best: 0.5065798 (2000)	tota

 20%|█████████████████▌                                                                      | 1/5 [00:41<02:46, 41.52s/it]

...

Learning rate set to 0.072699
0:	learn: 9.2604847	test: 9.2566518	best: 9.2566518 (0)	total: 7.26ms	remaining: 36.3s
200:	learn: 1.1609247	test: 1.1761170	best: 1.1761170 (200)	total: 998ms	remaining: 23.8s
400:	learn: 0.9128861	test: 0.9284839	best: 0.9284839 (400)	total: 1.99s	remaining: 22.8s
600:	learn: 0.7829628	test: 0.7987530	best: 0.7987530 (600)	total: 3.02s	remaining: 22.1s
800:	learn: 0.7008547	test: 0.7167152	best: 0.7167152 (800)	total: 4.06s	remaining: 21.3s
1000:	learn: 0.6401249	test: 0.6568936	best: 0.6568936 (1000)	total: 5.15s	remaining: 20.6s
1200:	learn: 0.5963700	test: 0.6141095	best: 0.6141095 (1200)	total: 6.09s	remaining: 19.3s
1400:	learn: 0.5601521	test: 0.5785666	best: 0.5785666 (1400)	total: 7.09s	remaining: 18.2s
1600:	learn: 0.5308033	test: 0.5500638	best: 0.5500638 (1600)	total: 8.15s	remaining: 17.3s
1800:	learn: 0.5059267	test: 0.5259758	best: 0.5259758 (1800)	total: 9.22s	remaining: 16.4s
2000:	learn: 0.4837249	test: 0.5045642	best: 0.5045642 (2000)	t

 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [02:38<00:39, 39.38s/it]

Learning rate set to 0.072699
0:	learn: 9.2625918	test: 9.2370575	best: 9.2370575 (0)	total: 5.99ms	remaining: 29.9s
200:	learn: 1.1662350	test: 1.1716899	best: 1.1716899 (200)	total: 1.09s	remaining: 26.1s
400:	learn: 0.9129059	test: 0.9214787	best: 0.9214787 (400)	total: 2.12s	remaining: 24.3s
600:	learn: 0.7835346	test: 0.7945985	best: 0.7945985 (600)	total: 3.14s	remaining: 23s
800:	learn: 0.7031019	test: 0.7154806	best: 0.7154806 (800)	total: 4.2s	remaining: 22s
1000:	learn: 0.6413422	test: 0.6549456	best: 0.6549456 (1000)	total: 5.27s	remaining: 21.1s
1200:	learn: 0.5985152	test: 0.6134305	best: 0.6134305 (1200)	total: 6.28s	remaining: 19.9s
1400:	learn: 0.5621220	test: 0.5778599	best: 0.5778599 (1400)	total: 7.29s	remaining: 18.7s
1600:	learn: 0.5335300	test: 0.5505419	best: 0.5505419 (1600)	total: 8.32s	remaining: 17.7s
1800:	learn: 0.5090980	test: 0.5272940	best: 0.5272940 (1800)	total: 9.42s	remaining: 16.7s
2000:	learn: 0.4860141	test: 0.5052893	best: 0.5052893 (2000)	total:

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 17.03it/s]


cos_sim with best submission: 0.9999547832446889
